# CW2.1:  Interpreter for FUNC

This week's assignment consists of two parts: 
- Writing an interpreter for the parts not requiring functions (*6 points*)
- Writing an interpreter for the parts requiring functions  (*4 points*)

In [1]:
type exp = Numb of int | Id of string | App of string * exp list

type bop = Less | LessEq | Eq | NEq 
type cond = C of bop * exp * exp

type statement =
  Assign of string * exp
| Read of string
| Write of exp 
| If of cond * statement list
| Ite of cond * statement list * statement list
| While of cond * statement list

type mmethod = M of string (* name of function *)
                * string list (* arguments *)
                * string list (* declarations *) 
                * statement list (* function body *)
                * string option (* possible return value *)

type program = P of mmethod list

type exp = Numb of int | Id of string | App of string * exp list


type bop = Less | LessEq | Eq | NEq


type cond = C of bop * exp * exp


type statement =
    Assign of string * exp
  | Read of string
  | Write of exp
  | If of cond * statement list
  | Ite of cond * statement list * statement list
  | While of cond * statement list


type mmethod =
    M of string * string list * string list * statement list * string option


type program = P of mmethod list


As in the lectures, you can use environments to look up variables. 
As there are no block statements, a simple dictionary suffices.

In [2]:
exception RuntimeError of string

(* This will define maps with strings as key *)
module Env = Map.Make(String)

(* Env.empty denotes the empty environment. 
We can add elements to an environment via Env.add.
This is the environment which only binds “a” to 3. *)
let example_env = Env.add "a" 3 Env.empty;;

(* We can look up elements in an environment via Env.find.
Env.find throws an exception if the key does not exist.*)
Env.find "a" example_env;;

let update_env (x : string) v env = Env.add x v env 
let lookup_env (x : string) env = Env.find x env 

exception RuntimeError of string


module Env :
  sig
    type key = String.t
    type 'a t = 'a Map.Make(String).t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val filter_map : (key -> 'a -> 'b option) -> 'a t -> 'b t
    val partition : (key -> 'a -> bool) 

val example_env : int Env.t = <abstr>


- : int = 3


val update_env : string -> 'a -> 'a Env.t -> 'a Env.t = <fun>


val lookup_env : string -> 'a Env.t -> 'a = <fun>


## Part 1: Interpreter for Basic Parts (6 points)

In the first part of the assignment, you write an interpreter for the part that does not require arbitrary functions, i.e. the subpart of the syntax without the syntactic classes ``program``, ``methods``, ``method``, and ``args`` and with expressions restricted to: 


```
<exp> ::= <id> | plus (<exp>, <exp>) | minus (<exp>, <exp>) | times (<exp>, <exp>) | divide (<exp>, <exp>) | <int> 
```

For now, values are simply integers:

In [3]:
type value = VInt of int

type value = VInt of int


Write a mutually recursive evaluator for statements in FUNC. 

In [4]:
let rec eval_exp (e : exp) (env : value Env.t) : value = match e with 
    | Numb n -> VInt n
    | Id x -> lookup_env x env
    
    | App ("plus", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                    | VInt m, VInt n -> VInt (m + n)
                                    )            
                                
    | App ("times", [e1; e2]) -> (match (eval_exp e1 env, eval_exp e2 env) with
                                            | (VInt m, VInt n) -> VInt (m * n)
                                            ) 
                                                                                        
    | App ("minus", [e1; e2]) -> (match (eval_exp e1 env, eval_exp e2 env) with
                                            | (VInt m, VInt n) -> VInt (m - n)
                                            )
      
    | App ("divide", [e1; e2]) -> (match (eval_exp e1 env, eval_exp e2 env) with
                                            | (VInt m, VInt n) -> VInt (m / n)
                                            )
      
    | App (f, es) -> raise (RuntimeError "Arbitrary functions not implemented.") 

 and eval_condexp (e : cond) (env : value Env.t) : int = match e with                          
      | C (Less, e1, e2) -> (match (eval_exp e1 env, eval_exp e2 env) with
      | VInt m, VInt n -> if m < n then 1 else 0 ) 
      
      | C(LessEq, e1, e2) -> (match eval_exp e1 env, eval_exp e2 env with 
      | VInt m, VInt n -> if m <= n then 1 else 0 )
      
      | C(Eq, e1, e2) -> (match eval_exp e1 env, eval_exp e2 env with 
      | VInt m, VInt n -> if m = n then 1 else 0 )
            
      | C(NEq, e1, e2) -> (match eval_exp e1 env, eval_exp e2 env with 
      | VInt m, VInt n -> if m <> n then 1 else 0 )

       
 and eval_stmt (s : statement) (env : value Env.t) : value Env.t  = (match s with 
    (* TODO: Cases for Assign, If, Ite, While, Write *)
    
      | Assign (x, e) -> (match eval_exp e env with
          | v -> update_env x (eval_exp e env) env)
          
      | If (e, c) -> (match eval_condexp e env with
          | 1 -> eval_stmts c env
          | _ -> env)
          
      | Ite (e, c1, c2) -> (match eval_condexp e env with
          | 1 -> eval_stmts c1 env
          | 0 -> eval_stmts c2 env
          | _ -> env)
          
      | While (c, st) -> let rec loop env = let b = eval_condexp c env in if b = 1 
                                                      then loop (eval_stmts st env) 
                                                      else env in loop env
          
      | Write (e) -> (match eval_exp e env with
          |VInt num -> print_endline ("OUTPUT: " ^ string_of_int num); env)
          
      | Read _ -> raise (RuntimeError "Read not implemented"))   
                        
 and eval_stmts (stmts : statement list) (env : value Env.t) : value Env.t = (match stmts with 
    | [] -> env
    | s :: stms' -> eval_stmts stms' (eval_stmt s env))

val eval_exp : exp -> value Env.t -> value = <fun>
val eval_condexp : cond -> value Env.t -> int = <fun>
val eval_stmt : statement -> value Env.t -> value Env.t = <fun>
val eval_stmts : statement list -> value Env.t -> value Env.t = <fun>


**Optional hints.** Your evaluator should contain, mutually recursively: 
-  a function ``eval_exp : exp -> value Env.t -> value`` evaluating expressions to values. 
Your expressions should correctly evaluate identifiers ``Id x`` and constants ``Numb n``, but for function arguments have to only evaluate ``App ("plus", [e1; e2])``, ``App ("minus", [e1; e2])``, ``App ("times", [e1; e2])`` and ``App ("divide", [e1; e2])``.
You can match directly on these cases, i.e. 
```
match e with 
| App ("plus", [e1; e2]) -> (* What it should evaluate to *)
```
Note that in these cases, no methods have to be executed - ``App ("plus", [VInt m; VInt n])`` simply evaluates to ``VInt (m + n)``.
All other function calls should throw an error message saying that they are not implemented: 
```
match e with 
... 
| App (f, es) -> raise (RuntimeError "Arbitrary functions not implemented.")
```
(You want to test that all tests until ``test_add`` are working before you continue.)
- a function ``eval_condexp: cond -> value Env.t -> int`` that evaluates conditional expressions. 
- a function ``eval_stmt: statement -> value Env.t -> value Env.t`` that evaluates a single statement.
As in SIMP, please ignore the ``Read`` primitive, i.e. add the case: 

```
    | Read _ -> raise (RuntimeError "Read not implemented.")
```

- a function ``eval_stmts: statement list -> value Env.t -> value Env.t`` that evaluates a list of statements. 

### Test Cases Part 1
Here is a (not necessarily complete) set of test cases: 

In [1]:
(* let p1 : program  =
  P
    [M ("pow", ["x"; "y"], ["i"; "res"],
      [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")],
      Some "res");
     M ("main", [], ["a"; "b"; "x"],
      [Assign ("a", Numb 5); Assign ("b", Numb 2);
       Assign ("x", App ("pow", [Id "b"; Id "a"]));
       Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
      None)] *)

let env = update_env "x" (VInt 2) (update_env "y" (VInt 5) Env.empty)

let test_numb =  eval_exp (Numb 3) env
(* Should yield VInt 3. *)

let test_id =  eval_exp (Id "x") env
(* Should yield VInt 2. *)

let test_add = eval_exp (App ("plus", [Id "x"; Numb 3])) env
(* Should yield VInt 5. *)

let test_stmt_write = eval_stmt (Write (Numb 3)) env
(* This should yield: "OUTPUT: 3" *)

let test_stmts = eval_stmts [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")] env  
       
(* This should yield: "OUTPUT: 32". *)

error: compile_error

## Part 2: Interpreter Including Methods (4 points)

In the second part, you extend the interpreter with methods. 

Values can now be *either* integers *or* closures: 

In [6]:
type value = VInt of int | Closure of string list * statement list * string option 

type value =
    VInt of int
  | Closure of string list * statement list * string option


### Closures 

A **closure** describes a method, to be evaluated once we know what expressions the variables are bound to. It consists of the **argument names** ``string list``, a list of statements called the **function body** ``statement list`` and an optional return variable ``string option``.

**Example**. Let us look at the example of: 

```
method pow(x, y) vars i, res
begin
    res := x; 

    i := 1; 

    while less(i,y)
    begin
        res := times(res,x);
        i := plus(i,1); 
    endwhile;
    write res;
    return res;
endmethod;

method main() vars a, b, x
begin
    a := 5; b := 2; 

    x := pow(b,a);
    if  eq(x,32) then write 1; else write 0; endif; 
endmethod;
```

This is what happens during evaluation:
1. "pow" is bound to the closure of ``pow``, i.e. 
    ``Closure (["x"; "y"], "res := x ... write res", Some res)``. 
2. "main" is bound to the closure of ``main``, i.e. 
    ``Closure ([], "a := 5... endif", None)``.
3. The main function, ``App ("main", [])`` is evaluated in the enviroment ``env`` where pow/main are bound to their closure. As the main function has no arguments, no variables are bound to expressions. Evaluating ``App ("main", [])`` means that we 
    1. Look up the closure of "main" in the environment, ensuring it's actually a closure. (If it's not a closure, throw a Runtime Error.) 
    2. Potentially update the environment to bind the arguments of "main" to the handed over expressions. In this case, main does not take any arguments.
    3. Execute the statements of "main". Here, we first enhance the environment to env' and bind "a" to ``VInt 5`` and b to ``VInt 2``. Then, we call "pow(b, a)". 
        1. We look up the closure of "pow" in the environment, ensuring it's actually a closure. (If it's not a closure, throw a Runtime Error.)
        2. We update the environment to an environment env'' and bind "x" to the value of b, i.e. ``VInt 2``, and "y" to the value of "a", i.e. ``VInt 5``. This means the environment now contains "main", "pow", "a", "b", "x", and "y".
        3. We evaluate the statements of "pow" in this updated environment env'' to a new environment in which for example res is bound to 32. As res is the variable in the return statement, we evaluate to ``VInt 32`` in this most recent environment.
    4. We bind "x" to ``VInt 32``, and continue with the environment before calling ``pow(b, a)``, i.e. env'. 
    5. We write 1.

In [7]:
(* TODO: Copy your solution from part 1. 
You will have to change the cases where you match on values, i.e. 

| App ("plus", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                | VInt m, VInt n -> VInt (m + n))


Also, you will have to adapt the case of arbitary functions.
*)

let rec eval_exp (e : exp) (env : value Env.t) : value = match e with 
    | Numb n -> VInt n
    | Id x -> lookup_env x env
    
    | App ("plus", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                    | VInt m, VInt n -> VInt (m + n)
                                    | _ -> raise (RuntimeError "Incorrect args!")
                                    )            
                                
    | App ("times", [e1; e2]) -> (match (eval_exp e1 env, eval_exp e2 env) with
                                            | (VInt m, VInt n) -> VInt (m * n)
                                            | _ -> raise (RuntimeError "Incorrect args!")
                                            ) 
                                                                                        
    | App ("minus", [e1; e2]) -> (match (eval_exp e1 env, eval_exp e2 env) with
                                            | (VInt m, VInt n) -> VInt (m - n)
                                            | _ -> raise (RuntimeError "Incorrect args!")
                                            )
      
    | App ("divide", [e1; e2]) -> (match (eval_exp e1 env, eval_exp e2 env) with
                                            | (VInt m, VInt n) -> VInt (m / n)
                                            | _ -> raise (RuntimeError "Incorrect args!")
                                            )
                                                  
    | App (f, es) -> let c = eval_exp (Id f) env in
                        (match c with
                                | Closure (args, s, ret) when List.length args = List.length es ->
                                    let env' = List.fold_left2
                      (fun acc arg exp -> update_env arg (eval_exp exp env) acc)
                          env
                          args
                          es
                        in
                        let env'' = eval_stmts s env' in
                        (match ret with
                            | None -> VInt 0
                            | Some x -> eval_exp (Id x) env''
                        )
                 | _ -> raise (RuntimeError "Invalid number of arguments or not a function value")
              )
              
              
 and eval_condexp (e : cond) (env : value Env.t) : int = match e with                          
      | C (Less, e1, e2) -> (match (eval_exp e1 env, eval_exp e2 env) with
      | VInt m, VInt n -> if m < n then 1 else 0  
      | _ -> raise (RuntimeError "Invalid Comparison"))
      
      | C(LessEq, e1, e2) -> (match eval_exp e1 env, eval_exp e2 env with 
      | VInt m, VInt n -> if m <= n then 1 else 0 
      | _ -> raise (RuntimeError "Invalid Comparison"))
      
      | C(Eq, e1, e2) -> (match eval_exp e1 env, eval_exp e2 env with 
      | VInt m, VInt n -> if m = n then 1 else 0 
      | _ -> raise (RuntimeError "Invalid Comparison"))
            
      | C(NEq, e1, e2) -> (match eval_exp e1 env, eval_exp e2 env with 
      | VInt m, VInt n -> if m <> n then 1 else 0 
      | _ -> raise (RuntimeError "Invalid Comparison"))

       
 and eval_stmt (s : statement) (env : value Env.t) : value Env.t  = (match s with 
    (* TODO: Cases for Assign, If, Ite, While, Write *)
    
      | Assign (x, e) -> (match eval_exp e env with
          | v -> update_env x (eval_exp e env) env)
          
      | If (e, c) -> (match eval_condexp e env with
          | 1 -> eval_stmts c env
          | _ -> env)
          
      | Ite (e, c1, c2) -> (match eval_condexp e env with
          | 1 -> eval_stmts c1 env
          | 0 -> eval_stmts c2 env
          | _ -> env)
          
      | While (c, st) -> let rec loop env = 
                              let b = eval_condexp c env in 
                              if b = 1 
                              then loop (eval_stmts st env) 
                              else env in loop env
          
       | Write (e) -> (match eval_exp e env with
          |VInt num -> print_endline ("OUTPUT: " ^ string_of_int num); env
          | _ -> raise (RuntimeError "Invalid Write Expression")) 
          
          
       | Read _ -> raise (RuntimeError "Read not implemented"))   
                        
 and eval_stmts (stmts : statement list) (env : value Env.t) : value Env.t = (match stmts with 
    | [] -> env
    | s :: stms' -> eval_stmts stms' (eval_stmt s env))
                        
let eval_method (m : mmethod) (env : value Env.t) = match m with 
    | M (f, args, dcls, s, ret) ->  let cl = Closure (args, s, ret) in update_env f cl env
    
let rec eval_methods (cs : mmethod list) (env : value Env.t) : value Env.t = match cs with 
    | [] -> env
    | c :: cs -> let new_env = eval_method c env in eval_methods cs new_env
    
let eval_program p = match p with 
    | P ms -> let env_with_methods = eval_methods ms Env.empty in eval_exp (App("main",[]) ) env_with_methods
    | P [] -> raise (RuntimeError "No methods found!")

val eval_exp : exp -> value Env.t -> value = <fun>
val eval_condexp : cond -> value Env.t -> int = <fun>
val eval_stmt : statement -> value Env.t -> value Env.t = <fun>
val eval_stmts : statement list -> value Env.t -> value Env.t = <fun>


val eval_method : mmethod -> value Env.t -> value Env.t = <fun>


val eval_methods : mmethod list -> value Env.t -> value Env.t = <fun>


val eval_program : program -> value = <fun>


**Implementation Hints.** Your evaluator should do the following modifications to the code in part 1:

- You will want to start by copying your code from part 1.
- You might have to change parts where you are matching on values as functions might also be closures.
- You require a function ``eval_method : mmethod -> value Env.t -> value Env.t`` that adds the closure corresponding to the method to the environment. Note that *nothing* is evaluated at this point, but we just do a binding, i.e. ``eval_method (M ("pow", ["x"; "y"], ["i"; "res"], [Assign ("res", Id "x") ...], Some "res"))`` should yield the updated environment where "pow" is bound to ``Closure (["x"; "y"], [Assign ("res", Id "x") ...], Some res)``.

- You require a function ``eval_methods : mmethod list -> value Env.t -> value Env.t`` that repeats the above step to a list of methods. 
(Test that ``test_method`` and ``test_methods`` are working before you continue.)

- The function ``eval_program : program -> value`` should first evaluates the list of methods, and then evaluates `` eval_exp (App ("main", [])) env`` in this updated environment.

- You have to modify ``eval_exp`` for applications: 

```
 | App (f, es) -> ...
```
    
You might want to first start by getting this to work if ``es`` is empty. 
In this case you simply evaluate the body of *f*. 
(Test that ``test_no_arguments`` is working.)

- In the case of arguments, you have to first evaluate all expressions ``es`` (you can get a list of values using the ``List.map`` primitive, ``List.map (fun x -> eval_exp x env) es`` which evaluates all elements in ``es`` in the environment ``env``). You then have to bind these values to the arguments of "f", which you can find in its closure, to a new environment env'. You might require a new function for this part.
You then evaluate the body of f, i.e. use ``eval_stmts`` on ``es`` in the new environment env'.

### Test Cases Part 2
Here is a (not necessarily complete) set of test cases: 

In [8]:
let pow = M ("pow", ["x"; "y"], ["i"; "res"],
      [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")],
      Some "res") 
      
let methods = [M ("pow", ["x"; "y"], ["i"; "res"],
      [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")],
      Some "res");
     M ("main", [], ["a"; "b"; "x"],
      [Assign ("a", Numb 5); Assign ("b", Numb 2);
       Assign ("x", App ("pow", [Id "b"; Id "a"]));
       Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
      None)]

let test_method = let 
        env = eval_method pow Env.empty in 
        assert (lookup_env "pow" env = Closure (["x"; "y"], [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")], Some "res"))

let test_methods = let
        env = eval_methods methods Env.empty in 
        assert (lookup_env "pow" env = Closure (["x"; "y"], [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")], Some "res"))


let prog_no_arg = P [M ("main", [], ["a"; "b"; "x"],
      [Assign ("a", Numb 5); Assign ("b", Numb 2);
       Assign ("x", App ("plus", [Id "b"; Id "a"]));
       Write (Id "x")],
      None)]


let test_no_argument = eval_program prog_no_arg
(* Should print: "OUTPUT: 7." *)

let p1 : program  =
  P
    [M ("pow", ["x"; "y"], ["i"; "res"],
      [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")],
      Some "res");
     M ("main", [], ["a"; "b"; "x"],
      [Assign ("a", Numb 5); Assign ("b", Numb 2);
       Assign ("x", App ("pow", [Id "b"; Id "a"]));
       Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
      None)]
      
let test_program = eval_program p1      
(* Should print: 
OUTPUT:32
OUTPUT:1 *)

val pow : mmethod =
  M ("pow", ["x"; "y"], ["i"; "res"],
   [Assign ("res", Id "x"); Assign ("i", Numb 1);
    While (C (Less, Id "i", Id "y"),
     [Assign ("res", App ("times", [Id "res"; Id "x"]));
      Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
    Write (Id "res")],
   Some "res")


val methods : mmethod list =
  [M ("pow", ["x"; "y"], ["i"; "res"],
    [Assign ("res", Id "x"); Assign ("i", Numb 1);
     While (C (Less, Id "i", Id "y"),
      [Assign ("res", App ("times", [Id "res"; Id "x"]));
       Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
     Write (Id "res")],
    Some "res");
   M ("main", [], ["a"; "b"; "x"],
    [Assign ("a", Numb 5); Assign ("b", Numb 2);
     Assign ("x", App ("pow", [Id "b"; Id "a"]));
     Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
    None)]


val test_method : unit = ()


val test_methods : unit = ()


val prog_no_arg : program =
  P
   [M ("main", [], ["a"; "b"; "x"],
     [Assign ("a", Numb 5); Assign ("b", Numb 2);
      Assign ("x", App ("plus", [Id "b"; Id "a"])); Write (Id "x")],
     None)]


OUTPUT: 7


val test_no_argument : value = VInt 0


val p1 : program =
  P
   [M ("pow", ["x"; "y"], ["i"; "res"],
     [Assign ("res", Id "x"); Assign ("i", Numb 1);
      While (C (Less, Id "i", Id "y"),
       [Assign ("res", App ("times", [Id "res"; Id "x"]));
        Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
      Write (Id "res")],
     Some "res");
    M ("main", [], ["a"; "b"; "x"],
     [Assign ("a", Numb 5); Assign ("b", Numb 2);
      Assign ("x", App ("pow", [Id "b"; Id "a"]));
      Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
     None)]


OUTPUT: 32
OUTPUT: 32
OUTPUT: 1


val test_program : value = VInt 0
